<a href="https://colab.research.google.com/github/omanshuthapliyal/ML2Coursework/blob/master/Omanshu_Thapliyal_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Student Name: Omanshu Thapliyal
#ECE 595 Machine Learning II
#Project 3: GAN - Student Code

In [0]:
#Import necessary packages
import numpy as np
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from keras.datasets import mnist
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import adam
from keras.models import load_model
import matplotlib.pyplot as plt

Using TensorFlow backend.


#Part 1: Implementing the GAN

In [0]:
#Load MNIST data and normalize to [-1, 1]
(data_train, _), (data_test, _) = mnist.load_data()
data_train = 2*(data_train/255.0 - 0.5)
data_test = 2*(data_test/255.0 - 0.5)
print(data_test.min(),data_test.max())


11501568/11490434 [==============================] - 1s 0us/step
(-1.0, 1.0)


In [0]:
# The D-dimensional noise vector length
latent_dim = 100
data_dim = 784

# Optimizer for discriminator, which will have a higher learning rate than adversarial model
def adam_optimizer():
    return adam(lr = 0.002, beta_1 = 0.5, beta_2 = 0.999)

# Optimizer for generator, which will have a lower learning rate than adversarial model
def gan_optimizer():
    return adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)

In [0]:
# Genrerator model
def create_generator():
  # Input layer: a 100-dimensional noise vector
  generator = Sequential()
  generator.add(Dense(100, input_dim = latent_dim))
  # First hidden layer: a dense (fully-connected) layer consisting of 200 - 300 units with a LeakyReLU activation
  generator.add(Dense(250, activation=LeakyReLU(0.5)) )
  # Second hidden layer: a dense (fully-connected) layer consisting of 500 - 600 units with a LeakyReLU activation
  generator.add(Dense(550, activation=LeakyReLU(0.5)) )
  # Third hidden layer: a dense (fully-connected) layer consisting of 1000 - 1200 units with a LeakyReLU activation
  generator.add(Dense(1100, activation=LeakyReLU(0.5)) )
  # Output layer: the 784-dimensional generated image with a hyperbolic tangent activation
  generator.add(Dense(data_dim, activation = 'tanh'))
  # Compiling model
  generator.compile(loss = 'mean_squared_error', 
                    Optimizer = adam_optimizer())

In [0]:
# Discriminator model
def create_discriminator():
  discriminator = Sequential()
  # Input layer: a 784-dimensional vector of the image data
  discriminator.add(Dense(784))
  # First hidden layer: a dense (fully-connected) layer consisting of 1000 - 1200 units with a LeakyReLU activation (add a ‘Dropout’ layer to avoid overfitting)
  discriminator.add(Dense(1100, activation=LeakyReLU(0.5)) )
  # Second hidden layer: a dense (fully-connected) layer consisting of 500 - 600 units with a LeakyReLU activation (add a ‘Dropout’ layer to avoid overfitting)
  discriminator.add(Dense(550, activation=LeakyReLU(0.5)) )
  # Third hidden layer: a dense (fully-connected) layer consisting of 200 - 300 units with a LeakyReLU activation
  discriminator.add(Dense(250, activation=LeakyReLU(0.5)) )
  # Output layer: a single Dense unit with a sigmoid activation indicating whether the image is ‘real’ or ‘fake’
  discriminator.add(Dense(units=1, activation = 'sigmoid'))
  discriminator.compile(loss='mean_squared_error', 
                        Optimzer = adam_optimizer(), 
                        metrics = ['accuracy'])

  return discriminator

10000

In [0]:
# GAN model
def create_gan(discriminator, generator):
  # FILL THIS IN
  discriminator.trainable = False
  gan_input = Input(shape = (latent_dim,))
  x = generator(gan_input)
  gan_output = discriminator(x)
  gan = Model(inputs = gan_input, Output = gan_output)
  gan.compile(loss = 'mean_squared_error', 
              optimizer = gan_optimizer(), 
              metrics = ['accuracy'])
  return gan

  

In [0]:
# Creating Graph for GAN
generator = create_generator()
discriminator = create_discriminator()
gan = create_gan(discriminator, generator)

# Model and training parameters
epochs = 5 #10000
batch_size = 50
sample_interval = 5 #10000

# Array to save training history
training_meta_data = np.zeros([epochs, 4])

TypeError: ignored

In [0]:
# Training the GAN
for e in range(1, epochs+1):
    # Generate random noise as input
    N = batch_size*latent_dim
    rand_input = np.random.multivariate_normal(mean = np.zeros(N, 1),
                                               cov = np.eye(N,N))
    rand_input = rand_input.reshape((batch_size, latent_dim))
    # Generate fake MNIST images from generated noise
    fake_images = generator(rand_input)
    # Get a random set of real MNIST images
    real_images = np.random.choice(data_train, size = batch_size)
    # Concatenate real and fake images into a single array (or batch)
    data_total = np.concatenate((real_images, fake_images))
    # Assign training labels (assign high probability, but not 1, to real images)
    labels_real = np.ones(real_images.shape[0])
    labels_fake = np.ones(fake_images.shape[0])
    labels_discriminator = np.concatenate(labels_real, labels_fake)

    # Allow discriminator parameters to be updated
    discriminator.trainable = True
    # Train discriminator on batch of real and fake images. Assign loss and accuracy to variable
    discriminator.train(data_total, labels_discriminator)

    # Train adversarial model and try to fool discriminator (with incorrect label) 
    # by generating a new batch of noise and assign them labels of real data
    

    # Keep discriminator weights constant while training generator
    discriminator.trainable = False

    # Train GAN (without updating discriminator weights) on new batch of fake images. Assign loss and accuracy to variable
    generator.train(data_total, labels_discriminator)

    # Save training status
    # Discriminator and model loss
    training_meta_data[e-1, 0] = d_loss[0]
    training_meta_data[e-1, 1] = gan_loss[0]

    # Discriminator and model accuracy
    training_meta_data[e-1, 2] = d_loss[1]
    training_meta_data[e-1, 3] = gan_loss[1]


    # If at sample interval, print training status and save samples
    if e % sample_interval == 0:
      
        # Print training status
        print("Epoch %d" %e)
        log_mesg = "%d: [Discriminaotr loss: %f, acc: %f]" % (e, d_loss[0], d_loss[1])
        log_mesg = "%s  [GAN loss: %f, acc: %f]" % (log_mesg, gan_loss[0], gan_loss[1])
        print(log_mesg)
        
        # Plot images 
        r, c = 5, 5

        # Create images from the noise (predict the outcome of the noise)
        gen_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow((gen_imgs[cnt].reshape(28, 28)), cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        plt.show()

In [0]:
# Plot model loss vs epoch
#FILL THIS CODE BLOCK IN AND PRODUCE PLOT

In [0]:
# Plot accuracy vs epoch
#FILL THIS CODE BLOCK IN AND PRODUCE PLOT

Answer the following questions:



1.   Why does the accuracy of the discriminator remain around 50%? Is this a good trait of the GAN? 

  ANS: 


2.   How could this model be modified to produce cleaner (less noisy) images? 

  ANS: 

#Part 2: Generating samples using trained generator

In [0]:
# Generate ten images from Gaussian noise using the trained generator from Part 1
# FILL THIS IN

# Re-scale generated images to lie in [0, 1]
# FILL THIS IN

In [0]:
# Visualize generated noise
r, c = 2, 5
fig, axs = plt.subplots(r, c)
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow((noise[cnt].reshape(10, 10)), cmap='gray')
        axs[i,j].axis('off')
        cnt += 1
plt.show()

In [0]:
# Visualize generated samples
r, c = 2, 5
fig, axs = plt.subplots(r, c)
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow((generated_images[cnt].reshape(28, 28)), cmap='gray')
        axs[i,j].axis('off')
        cnt += 1
plt.show()

#Part 3: Testing accuracy of generated images on ten samples

In [0]:
# Load mnist classifier and generated images
mnist_classifier = load_model('mnist_classifier.h5')

In [0]:
# ASSIGN CLASSES
labels = []

# Convert integer labels to one-hot labels 
labels = keras.utils.np_utils.to_categorical(labels, num_classes=10)

# Show classifications
# FILL THIS IN 

# Evaluate accuracy
# FILL THIS IN 